In [9]:
import os
import click 
import multiprocessing 
import glob
import pandas as pd 
from Bio import SeqIO
KOFAMSCAN_exe = '/mnt/storage3/yfdai/download/kofamscan/bin/exec_annotation'
KOFAMSCAN_profiles = '/mnt/home-db/pub/protein_db/kegg/v20230301/profiles'
KOFAMSCAN_ko_list = '/mnt/home-db/pub/protein_db/kegg/v20230301/ko_list'

In [ ]:
#integrate file for annotation
def integrate_faagbk(input_folder,target_folder):
    try:
        os.makedirs(target_folder, exist_ok=True)
    except OSError as err:
        if err.errno == os.errno.EEXIST and os.path.isdir(folder_path):
            pass
        else:
            raise
    

In [ ]:
###now start annotation
### use dys pipeline now(need to be fixed)
#python3 /mnt/storage3/yfdai/material/dysfunctional_dectector/raw_workflow/s1.annotation.py -fi /mnt/storage3/yfdai/material/ruegeria/GNM000011965_result/GNM000011965.faa /mnt/storage3/yfdai/material/ruegeria/GNM000011965_result/GNM000011965.gbk -o /mnt/storage3/yfdai/new_op
def run_annotation(target_folder,faa,gbk):
    wd = target_folder
    os.system(f"python3 /mnt/storage3/yfdai/material/dysfunctional_dectector/raw_workflow/s1.annotation.py -fi {faa} {gbk} -o {wd}/annotation_result")
    
def kofamscan_annotation(target_folder,pros):
    wd = target_folder
    try:
        os.makedirs(f"{wd}/annotation_result", exist_ok=True)
    except OSError as err:
        if err.errno == os.errno.EEXIST and os.path.isdir(folder_path):
            pass
        else:
            raise
    params=[]
    pattern = os.path.join(f"{wd}/preflow/prokka_result", '**', '*.faa')
    faa_files = glob.glob(pattern, recursive=True)
    pattern = os.path.join(f"{wd}/preflow/prokka_result", '**', '*.gbk')
    gbk_files = glob.glob(pattern, recursive=True)
    for faapath in faa_files:
        dict_name=faapath.split("/")[-2]
        for gbkpath in gbk_files:
            dict_n=gbkpath.split("/")[-2]
            if dict_n == dict_name:
                params.append((wd,faapath,gbkpath))
#prokka --compliant --cpus 4 --outdir /mnt/storage3/yfdai/material/ruegeria/GNM000011965_result --prefix GNM000011965 --locustag GNM000011965 /mnt/storage3/yfdai/material/ruegeria/GNM000011965.fna
    with multiprocessing.Pool(processes=pros) as pool:
        pool.starmap(run_annotation,params)
        pool.close
        pool.join

In [ ]:
kofamscan_annotation(2)

In [ ]:
###refine annotation
def get_refined_annotation(in_folder,out_folder):
    os.system(f"python3 /mnt/storage3/yfdai/material/dysfunctional_dectector/raw_workflow/s2.refine_annotation.py -o {out_folder}/s2out mlworkflow -i {in_folder} -nt 25 -rp")
#another account
#python3 /mnt/storage3/yfdai/material/dysfunctional_dectector/raw_workflow/s2.refine_annotation.py -o /mnt/storage3/yfdai/material/gene_loss/s2out mlworkflow -i /mnt/storage3/yfdai/material/gene_loss/annotation_result -nt 25 -rp

In [13]:
###merge the result of s1
#,fnn_file
def get_refined_pf_sequence_one(pos_file,fnn_file,target_file):
    container={}
    df = pd.read_csv(pos_file, sep='\t')
    filtered_df = df[df['pseudogenized'] != 'NOT']
    psgene_list = filtered_df.iloc[:, 0].tolist()
    with open(fnn_file, 'r') as file,open(target_file, 'w') as outfile:
        for record in SeqIO.parse(file, 'fasta'):
            if record.id in psgene_list:
                container[record.id]=record
        ordered_container = {key: container[key] for key in psgene_list if key in container}
        for seq_id,sequence in ordered_container.items():
            SeqIO.write(sequence, outfile, 'fasta')

def get_pf_sequence_all(pos_folder,fnn_folder,target_folder):
    try:
        os.makedirs(target_folder, exist_ok=True)
    except OSError as err:
        if err.errno == os.errno.EEXIST and os.path.isdir(target_folder):
            pass
        else:
            raise
    for filename in os.listdir(pos_folder):
        pos_name=filename.rsplit("_",1)[0]
        pos_path = os.path.join(pos_folder, filename)
        for fnn_file in os.listdir(fnn_folder):
            fnn_name=fnn_file.rsplit(".",1)[0]
            fnn_path = os.path.join(fnn_folder, fnn_file)
            if fnn_name==pos_name:
                get_refined_pf_sequence_one(pos_path,fnn_path,f"{target_folder}/{fnn_name}.fnn")
    
#def find_more_pseudos():
#    os.system(f"python3 /mnt/storage3/yfdai/material/dysfunctional_dectector/raw_workflow/s2.refine_annotation.py -o /mnt/storage3/yfdai/material/gene_loss/merge_result mlworkflow -i /mnt/storage3/yfdai/material/gene_loss/annotation_result -nt 20 -rp")
#get_refined_pf_sequence_one("/mnt/storage3/yfdai/new_op/pos/CE2_pos.tsv","/mnt/maple/ymchen/gene_loss/preflow/prokka_result/CE2/CE2.ffn","/mnt/maple/ymchen/gene_loss/CE2_pseudo.fnn")    